In [7]:
# Copyright 2018 Anar Amirli
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Feature Construction

### What dataset do we use?

In this document, given an entire 2017-2018 season's worth of player data from the Turkish professional soccer league, we generate input (aligning player trajectories based in role and then setting new features based on spatiotemporal data) and output data in order to build model for predicting ball location on the pitch.
Due to losses in data, some matches have been extracted, so the total number of matches used is 298.

### What is our approach?

Our approach is based on the player's role-assignment, rather than the player's individual data. We're given huge set of player spatiotemportal tracking data. In oreder to build our features set, we should arrange all these data in some particular order, so machine should understand what data it does use. We can not randomly pick player's data and use it in data construction. Therefore, using players individual data for analyzing lacks some important contextual concepts. Football is played by 11 players on each side, but there is always an open door for 3 substitution players. We can not also use players' data those aren't in the game, this should cause a lot of noise in the system. For doing so, we developed the role-based analysis.

As in all games, football also has its standard rules for assigning each player role, such as; the rearmost players are defence players, midfielders are players whose position of play is midway, forwards player are players on the team who play nearest to the opposing. Using this information we can assig role to each player.

1. Define which player's in the game.
2. Generate players' average positions based on their position in the timeline of previous 15 minutes.
3. Each minute update their average position.
4. Scale average positions to the range of (0-105)x(0-68) meters based on their standard deviation of distributional data.
5. Assing role to each player.


Sometimes we lost player's data for short period of time (0-15 seconds), even though he is still in the game (not substituted or set-off). This happens when tagger misses to tag data, the player receives medical treatment out of the pitch, or game is stopped (in this case all players data is null). In these praticualr cases we allply below approach.   


1. Chenck wheter player is still in game or not (not substituted or set-off).
2. If player's in game check wheter you're receiving his data or not.
3. If we receive his data then use it in the feature construction.

### Computing Features For a Single Match

In this section, we compute __*333*__ features in total for a single match. Each feature is computed for each second of the game. Thus, we iterate over the seconds of a match and compute each feature per second.

__Referee features:__ *(5 features)*
1. Coordinates $(x,y)$ 
2. Speed and its directions both on the x-axis and y-axis $\vec{V}$, $direction$ (1: along the axis, -1: opposite of the axis, 0: no direction change)

__Teams features:__ *(2 features)*
1. Average coordinates of denisity clustering $(x,y)_{DBSCAN}$
2. Teams average speed $\vec{V}_{avrg}$

__Home and Away team features:__ *(9 features)*
1. Average coordinates of denisity clustering $(x,y)_{DBSCAN}$
2. Team max and min coordinates $(x,y)_{max}$,$(x,y)_{min}$
3. Average team coordinates $(x,y)_{avg}$ 
4. Average team speed $\vec{V}_{avrg}$

__Goalkeeper features:__ *(3 features)*
1. Coordinates $(x,y)$ 
2. Speed $\vec{V}$ 

__Group features:__ *(16 features)*<br>

Note: ($\vec{V}_{Slow}\leq1.5$,&emsp;$ 1.5\lt\vec{V}_{hir}\leq3$,&emsp;$\vec{V}_{sprint}\gt3$)
1. Group max and min coordinates $(x,y)_{max}$,$(x,y)_{min}$
2. Group average coordinates $(x,y)_{avg}$
3. Group average speed  $\vec{V}_{avg}$
4. Average coords of the SLOW group $(x,y)_{slow}$
5. X-Density of slow group $density_{slow}$
6. Average coords of the HIR *(high intensity run)* group $(x,y)_{hir}$ 
7. X-Denisty of HIr group $density_{hir}$
8. Coords of the fastest player in the SPRINT group $(x,y)_{sprint_{max}}$
9. Speed of the fastest player in the SPRINT group and its directions both on the x-axis and y-axis $\vec{V}_{sprint_{max}}$

### Features

In [8]:
features = [

# Referee
'refX', 'refY', 'refSpeed', 'refDirectX', 'refDirectY',
    
    
# Team Features
'teamsAvgSpeed',                   'teamsMinSpeed',                   'teamMaxSpeed', 
'teamsMaxSprint',                  'teamsDBSCANx',                    'teamsDBSCANy',

# HOME TEAM
'homeDBSCANx',                     'homeDBSCANy',                     'homeMaxX',
'homeMinX',                        'homeMaxY',                        'homeMinY',
'homeAvgX',                        'homeAvgY',                        'homeAvgSpeed',
'homeMinSpeedX',                   'homeMinSpeedY',                   'homeMinSpeed',
'homeMaxSpeedX',                   'homeMaxSpeedY',                   'homeMaxSpeed',
'homeMaxSprintX',                  'homeMaxSprintY',                  'homeMaxSprint',

  
# Goalkeeper
'homeGoalKeeperX',                 'homeGoalKeeperY',                 'homeGoalKeeperSpeed',

# Left-Back
'homeLeftBackMaxX',                'homeLeftBackMinX',                'homeLeftBackMaxY',
'homeLeftBackMinY',                'homeLeftBackAvgX',                'homeLeftBackAvgY',
'homeLeftBackAvgSpeed',            'homeLeftBackSlowAvgX',            'homeLeftBackSlowAvgY',
'homeLeftBackHirAvgX',             'homeLeftBackHirAvgY',             'homeLeftBackMaxSprintX',
'homeLeftBackMaxSprintY',          'homeLeftBackMaxSprintSpeed',      'homeLeftBackMinSpeedX',
'homeLeftBackMinSpeedY',           'homeLeftBackMinSpeed',            'homeLeftBackMaxSpeedX',
'homeLeftBackMaxSpeedY',           'homeLeftBackMaxSpeed',
    
# Center-Back
'homeCenterBackMaxX',              'homeCenterBackMinX',              'homeCenterBackMaxY',
'homeCenterBackMinY',              'homeCenterBackAvgX',              'homeCenterBackAvgY',
'homeCenterBackAvgSpeed',          'homeCenterBackSlowAvgX',          'homeCenterBackSlowAvgY',
'homeCenterBackHirAvgX',           'homeCenterBackHirAvgY',           'homeCenterBackMaxSprintX',
'homeCenterBackMaxSprintY',        'homeCenterBackMaxSprintSpeed',    'homeCenterBackMinSpeedX',
'homeCenterBackMinSpeedY',         'homeCenterBackMinSpeed',          'homeCenterBackMaxSpeedX',
'homeCenterBackMaxSpeedY',         'homeCenterBackMaxSpeed',

# Right-Back
'homeRightBackMaxX',               'homeRightBackMinX',               'homeRightBackMaxY',
'homeRightBackMinY',               'homeRightBackAvgX',               'homeRightBackAvgY',
'homeRightBackAvgSpeed',           'homeRightBackSlowAvgX',           'homeRightBackSlowAvgY',
'homeRightBackHirAvgX',            'homeRightBackHirAvgY',            'homeRightBackMaxSprintX',
'homeRightBackMaxSprintY',         'homeRightBackMaxSprintSpeed',     'homeRightBackMinSpeedX',
'homeRightBackMinSpeedY',          'homeRightBackMinSpeed',           'homeRightBackMaxSpeedX',
'homeRightBackMaxSpeedY',          'homeRightBackMaxSpeed',

# Left-Winger
'homeLeftWingerMaxX',              'homeLeftWingerMinX',              'homeLeftWingerMaxY',
'homeLeftWingerMinY',              'homeLeftWingerAvgX',              'homeLeftWingerAvgY',
'homeLeftWingerAvgSpeed',          'homeLeftWingerSlowAvgX',          'homeLeftWingerSlowAvgY',
'homeLeftWingerHirAvgX',           'homeLeftWingerHirAvgY',           'homeLeftWingerMaxSprintX',
'homeLeftWingerMaxSprintY',        'homeLeftWingerMaxSprintSpeed',    'homeLeftWingerMinSpeedX',
'homeLeftWingerMinSpeedY',         'homeLeftWingerMinSpeed',          'homeLeftWingerMaxSpeedX',
'homeLeftWingerMaxSpeedY',         'homeLeftWingerMaxSpeed',
    
# Center-Middle
'homeCenterMiddleMaxX',            'homeCenterMiddleMinX',            'homeCenterMiddleMaxY',
'homeCenterMiddleMinY',            'homeCenterMiddleAvgX',            'homeCenterMiddleAvgY',
'homeCenterMiddleAvgSpeed',        'homeCenterMiddleSlowAvgX',        'homeCenterMiddleSlowAvgY',
'homeCenterMiddleHirAvgX',         'homeCenterMiddleHirAvgY',         'homeCenterMiddleMaxSprintX',
'homeCenterMiddleMaxSprintY',      'homeCenterMiddleMaxSprintSpeed',  'homeCenterMiddleMinSpeedX',
'homeCenterMiddleMinSpeedY',       'homeCenterMiddleMinSpeed',        'homeCenterMiddleMaxSpeedX',
'homeCenterMiddleMaxSpeedY',       'homeCenterMiddleMaxSpeed',
 
# Right-Winger
'homeRightWingerMaxX',             'homeRightWingerMinX',             'homeRightWingerMaxY',
'homeRightWingerMinY',             'homeRightWingerAvgX',             'homeRightWingerAvgY',
'homeRightWingerAvgSpeed',         'homeRightWingerSlowAvgX',         'homeRightWingerSlowAvgY',
'homeRightWingerHirAvgX',          'homeRightWingerHirAvgY',          'homeRightWingerMaxSprintX',
'homeRightWingerMaxSprintY',       'homeRightWingerMaxSprintSpeed',   'homeRightWingerMinSpeedX',
'homeRightWingerMinSpeedY',        'homeRightWingerMinSpeed',         'homeRightWingerMaxSpeedX',
'homeRightWingerMaxSpeedY',        'homeRightWingerMaxSpeed',

# Center-Forward
'homeCenterForwardMaxX',           'homeCenterForwardMinX',           'homeCenterForwardMaxY',
'homeCenterForwardMinY',           'homeCenterForwardAvgX',           'homeCenterForwardAvgY',
'homeCenterForwardAvgSpeed',       'homeCenterForwardSlowAvgX',       'homeCenterForwardSlowAvgY',
'homeCenterForwardHirAvgX',        'homeCenterForwardHirAvgY',        'homeCenterForwardMaxSprintX',
'homeCenterForwardMaxSprintY',     'homeCenterForwardMaxSprintSpeed', 'homeCenterForwardMinSpeedX',
'homeCenterForwardMinSpeedY',      'homeCenterForwardMinSpeed',       'homeCenterForwardMaxSpeedX',
'homeCenterForwardMaxSpeedY',      'homeCenterForwardMaxSpeed',



# AWAY TEAM
'awayDBSCANx',                     'awayDBSCANy',                     'awayMaxX',
'awayMinX',                        'awayMaxY',                        'awayMinY',
'awayAvgX',                        'awayAvgY',                        'awayAvgSpeed',
'awayMinSpeedX',                   'awayMinSpeedY',                   'awayMinSpeed',
'awayMaxSpeedX',                   'awayMaxSpeedY',                   'awayMaxSpeed',
'awayMaxSprintX',                  'awayMaxSprintY',                  'awayMaxSprint',
  
# Goalkeeper
'awayGoalKeeperX',                 'awayGoalKeeperY',                 'awayGoalKeeperSpeed',

# Left-Back
'awayLeftBackMaxX',                'awayLeftBackMinX',                'awayLeftBackMaxY',
'awayLeftBackMinY',                'awayLeftBackAvgX',                'awayLeftBackAvgY',
'awayLeftBackAvgSpeed',            'awayLeftBackSlowAvgX',            'awayLeftBackSlowAvgY',
'awayLeftBackHirAvgX',             'awayLeftBackHirAvgY',             'awayLeftBackMaxSprintX',
'awayLeftBackMaxSprintY',          'awayLeftBackMaxSprintSpeed',      'awayLeftBackMinSpeedX',
'awayLeftBackMinSpeedY',           'awayLeftBackMinSpeed',            'awayLeftBackMaxSpeedX',
'awayLeftBackMaxSpeedY',           'awayLeftBackMaxSpeed',
    
# Center-Back
'awayCenterBackMaxX',              'awayCenterBackMinX',              'awayCenterBackMaxY',
'awayCenterBackMinY',              'awayCenterBackAvgX',              'awayCenterBackAvgY',
'awayCenterBackAvgSpeed',          'awayCenterBackSlowAvgX',          'awayCenterBackSlowAvgY',
'awayCenterBackHirAvgX',           'awayCenterBackHirAvgY',           'awayCenterBackMaxSprintX',
'awayCenterBackMaxSprintY',        'awayCenterBackMaxSprintSpeed',    'awayCenterBackMinSpeedX',
'awayCenterBackMinSpeedY',         'awayCenterBackMinSpeed',          'awayCenterBackMaxSpeedX',
'awayCenterBackMaxSpeedY',         'awayCenterBackMaxSpeed',

# Right-Back
'awayRightBackMaxX',               'awayRightBackMinX',               'awayRightBackMaxY',
'awayRightBackMinY',               'awayRightBackAvgX',               'awayRightBackAvgY',
'awayRightBackAvgSpeed',           'awayRightBackSlowAvgX',           'awayRightBackSlowAvgY',
'awayRightBackHirAvgX',            'awayRightBackHirAvgY',            'awayRightBackMaxSprintX',
'awayRightBackMaxSprintY',         'awayRightBackMaxSprintSpeed',     'awayRightBackMinSpeedX',
'awayRightBackMinSpeedY',          'awayRightBackMinSpeed',           'awayRightBackMaxSpeedX',
'awayRightBackMaxSpeedY',          'awayRightBackMaxSpeed',

# Left-Winger
'awayLeftWingerMaxX',              'awayLeftWingerMinX',              'awayLeftWingerMaxY',
'awayLeftWingerMinY',              'awayLeftWingerAvgX',              'awayLeftWingerAvgY',
'awayLeftWingerAvgSpeed',          'awayLeftWingerSlowAvgX',          'awayLeftWingerSlowAvgY',
'awayLeftWingerHirAvgX',           'awayLeftWingerHirAvgY',           'awayLeftWingerMaxSprintX',
'awayLeftWingerMaxSprintY',        'awayLeftWingerMaxSprintSpeed',    'awayLeftWingerMinSpeedX',
'awayLeftWingerMinSpeedY',         'awayLeftWingerMinSpeed',          'awayLeftWingerMaxSpeedX',
'awayLeftWingerMaxSpeedY',         'awayLeftWingerMaxSpeed',
    
# Center-Middle
'awayCenterMiddleMaxX',            'awayCenterMiddleMinX',            'awayCenterMiddleMaxY',
'awayCenterMiddleMinY',            'awayCenterMiddleAvgX',            'awayCenterMiddleAvgY',
'awayCenterMiddleAvgSpeed',        'awayCenterMiddleSlowAvgX',        'awayCenterMiddleSlowAvgY',
'awayCenterMiddleHirAvgX',         'awayCenterMiddleHirAvgY',         'awayCenterMiddleMaxSprintX',
'awayCenterMiddleMaxSprintY',      'awayCenterMiddleMaxSprintSpeed',  'awayCenterMiddleMinSpeedX',
'awayCenterMiddleMinSpeedY',       'awayCenterMiddleMinSpeed',        'awayCenterMiddleMaxSpeedX',
'awayCenterMiddleMaxSpeedY',       'awayCenterMiddleMaxSpeed',
 
# Right-Winger
'awayRightWingerMaxX',             'awayRightWingerMinX',             'awayRightWingerMaxY',
'awayRightWingerMinY',             'awayRightWingerAvgX',             'awayRightWingerAvgY',
'awayRightWingerAvgSpeed',         'awayRightWingerSlowAvgX',         'awayRightWingerSlowAvgY',
'awayRightWingerHirAvgX',          'awayRightWingerHirAvgY',          'awayRightWingerMaxSprintX',
'awayRightWingerMaxSprintY',       'awayRightWingerMaxSprintSpeed',   'awayRightWingerMinSpeedX',
'awayRightWingerMinSpeedY',        'awayRightWingerMinSpeed',         'awayRightWingerMaxSpeedX',
'awayRightWingerMaxSpeedY',        'awayRightWingerMaxSpeed',

# Center-Forward
'awayCenterForwardMaxX',           'awayCenterForwardMinX',           'awayCenterForwardMaxY',
'awayCenterForwardMinY',           'awayCenterForwardAvgX',           'awayCenterForwardAvgY',
'awayCenterForwardAvgSpeed',       'awayCenterForwardSlowAvgX',       'awayCenterForwardSlowAvgY',
'awayCenterForwardHirAvgX',        'awayCenterForwardHirAvgY',        'awayCenterForwardMaxSprintX',
'awayCenterForwardMaxSprintY',     'awayCenterForwardMaxSprintSpeed', 'awayCenterForwardMinSpeedX',
'awayCenterForwardMinSpeedY',      'awayCenterForwardMinSpeed',       'awayCenterForwardMaxSpeedX',
'awayCenterForwardMaxSpeedY',      'awayCenterForwardMaxSpeed',
]

print(len(features))

333


In [39]:
pos_header = 'match_id,half,min,second,hgkx,hx1,hx2,hx3,hx4,hx5,hx6,hx7,hx8,hx9,hx10,hx11,hx12,hx13,hx14,hx15,hx16,hx17,hx18,hx19,hx20,hgky,hy1,hy2,hy3,hy4,hy5,hy6,hy7,hy8,hy9,hy10,hy11,hy12,hy13,hy14,hy15,hy16,hy17,hy18,hy19,hy20,agkx,ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8,ax9,ax10,ax11,ax12,ax13,ax14,ax15,ax16,ax17,ax18,ax19,ax20,agky,ay1,ay2,ay3,ay4,ay5,ay6,ay7,ay8,ay9,ay10,ay11,ay12,ay13,ay14,ay15,ay16,ay17,ay18,ay19,ay20'

In [10]:
%matplotlib inline
import matplotlib.pyplot as plt
from __future__ import division

import json
import pickle
import pandas as pd
import numpy as np
import math
from scipy.ndimage.interpolation import shift
import operator
import scipy as sc

# our common utility functions that are used in different notebooks
from utils import *


pd.options.display.max_columns = None
pd.set_option('compute.use_bottleneck', True)
pd.set_option('compute.use_numexpr', True)

### Computation

In [40]:
with open('../data/general/matches_2017_2018_v1.json') as matches_json:
    matches = json.load(matches_json)

minutes_step = 15 # timeline for calculating avrg pos

role_segment_coord = [[0, 40, 0, 19],    # Left Back 
                      [0, 35, 19, 49],   # Center Back
                      [0, 40, 49, 68],   # Right Back
                      [40, 105, 0, 19],  # Left Winger
                      [35, 75, 19, 49],  # Center Midfielder  
                      [40, 105, 49, 68], # Right Winger
                      [75, 105, 19, 49]] # Center Forward/Attacking Middle



# seeting feature set header
features_set_header = 'match_id,half,min,second,'
for f in features:
    features_set_header += f + ',' 
    
features_set_header += 'pitch_index,xpos,ypos,ball_owner'
    

# position group count set header
group_count_header = 'matchId,half,min,sec'

g_count_header = ['homeLeftBack',   'homeCenterBack',   'homeRightBack', 
                  'homeLeftWinger', 'homeCenterMiddle', 'homeRightWinger', 
                  'homeCenterForward', 
                  
                  'awayLeftBack',   'awayCenterBack',   'awayRightBack', 
                  'awayLeftWinger', 'awayCenterMiddle', 'awayRightWinger', 
                  'awayCenterForward']

for g in g_count_header:
    group_count_header += ',' + g
    
    
for match_data in matches:

    
    id = int(match_data['id'])
#     if id!=60617 and id!=60634 and id!=60696 and id!=60706 and id!=60821 and id!=60845 and id!=60664 and id!=116001326:
    if id==60565:
    
        print("\nMatch id:" + str(id))

        # defining 3 distinct id in game
        referee_id = 0
        home_team_id = int(match_data['homeId']) 
        away_team_id = int(match_data['awayId']) 

        # home and away player list
        home_team_player = []
        home_goal_keeper = []
        away_team_player = []
        away_goal_keeper = []
        
        
        ball_index_state=1
        
        
        refreee_pos_noise = True
        
        # all data
        all_data = []
        all_pos_data=[]
        all_data_count = []
        
        # players pos data
        players_pos_data = []

        # home_nonzero
        home_player_count_pitch = []
        home_prev_count = 11

        # home_nonzero
        away_player_count_pitch = []
        away_prev_count = 11

        # player dicts
        home_dict = {} 
        away_dict = {}
        
        # for detection noise data on x=0, y=0
        player_has_ball_detected = False
        
        # features vectors
        home_features_df = np.zeros(shape=(7, 20), dtype=np.float)
        away_features_df = np.zeros(shape=(7, 20), dtype=np.float)
        
        
        # ball location
        ball_location =  np.zeros(shape=(2), dtype=np.float)
        
        # match_info
        match_info =  np.zeros(shape=(4), dtype=np.int32)
        
        match_info[0]=id
        
        # teams speed 
        teams_speed =  np.zeros(shape=(1), dtype=np.float)
        # teams speed min,max,sprint data
        teams_speed_data =  np.zeros(shape=(3), dtype=np.float)
        
        
        # ball owner
        ball_owner =  np.zeros(shape=(1), dtype=np.float)
        

        # team features
        home_df = np.zeros(shape=(16), dtype=np.float)
        away_df = np.zeros(shape=(16), dtype=np.float)
        
        
        # features vectors count
        home_feature_count = np.zeros(shape=(7, 1), dtype=np.float)
        away_feature_count = np.zeros(shape=(7, 1), dtype=np.float)
        
        #features speed dict
        home_f_speed_dict = {0:{}, 1:{}, 2:{}, 3:{}, 4:{}, 5:{}, 6:{}}
        away_f_speed_dict = {0:{}, 1:{}, 2:{}, 3:{}, 4:{}, 5:{}, 6:{}}
        
        # features for Refreee and GKs
        refree_features = np.zeros(shape=(1,5), dtype=np.float)
        GK_home_features = np.zeros(shape=(1,3), dtype=np.float)
        GK_away_features = np.zeros(shape=(1,3), dtype=np.float)
        
        # ball position
        ball_pos_index = np.zeros(shape=(1), dtype=np.int8)

        # game duration
        whole_game_duration = 0

        # players position ID 
        player_positionId = {'homeTeam' : {}, 'awayTeam' : {}}

        with open('../data/match_' + str(id) + '/roster_data_' + str(id) + '.json', 'r') as file:
            match_squad = json.load(file)

        for roaster in match_squad:
            if roaster['teamId'] == home_team_id:
                home_team_player.append(str(roaster['jerseyNumber']))
                player_positionId['homeTeam'][roaster['jerseyNumber']] = roaster['positionId'] 
                if roaster['positionId'] == 1:
                    home_goal_keeper.append(str(roaster['jerseyNumber']))


            elif roaster['teamId'] == away_team_id:
                away_team_player.append(str(roaster['jerseyNumber']))
                player_positionId['awayTeam'][roaster['jerseyNumber']] = roaster['positionId']
                if roaster['positionId'] == 1:
                    away_goal_keeper.append(str(roaster['jerseyNumber']))


        # home players dict data
        home_team_player.sort(key=int)
        home_goal_keeper.sort(key=int)
        home_val = np.arange(0, len(home_team_player))
        home_dict = dict(zip(home_team_player, home_val))
        home_dict_reverse = dict(zip(home_val, home_team_player))


        # away players dict data
        away_team_player.sort(key=int)
        away_goal_keeper.sort(key=int)
        away_val = np.arange(0, len(away_team_player))
        away_dict = dict(zip(away_team_player, away_val))
        away_dict_reverse = dict(zip(away_val, away_team_player))

        # home and away team players' status[0: not playing, 1: playing, 2-left the game], 
        #enter status[0: enter 1st half, 1: enter second half],
        # enter time, and duration, red card
        home_dur_in_out = np.zeros(shape=(len(home_team_player), 5), dtype=np.float)
        away_dur_in_out = np.zeros(shape=(len(away_team_player), 5), dtype=np.float)

        
        
        #play positions:
        players_position_home_x = np.zeros(shape=(len(home_team_player)), dtype=np.float)
        players_position_home_y = np.zeros(shape=(len(home_team_player)), dtype=np.float)
        players_position_away_x = np.zeros(shape=(len(away_team_player)), dtype=np.float)
        players_position_away_y = np.zeros(shape=(len(away_team_player)), dtype=np.float)
        

        with open('../data/match_' + str(id) + '/players_data_' + str(id) + '.json', 'r') as file:
            player_data = json.load(file)


        for player in player_data:
            jersey_number = player['jerseyNumber']
            if player['teamId'] == home_team_id:
                home_dur_in_out[home_dict[str(jersey_number)]][3] = player['duration']
                home_dur_in_out[home_dict[str(jersey_number)]][4] = player['redCard']

            elif player['teamId'] == away_team_id:
                away_dur_in_out[away_dict[str(jersey_number)]][3] = player['duration']
                away_dur_in_out[away_dict[str(jersey_number)]][4] = player['redCard']

        # home and away team player pos and activity count: 0-xpos, 1-ypos
        home_pos_count = np.zeros(shape=(len(home_team_player), 2), dtype=np.float)
        home_activity_count = np.zeros(shape=(len(home_team_player), 1), dtype=np.int8)
        home_activity_count_all = np.zeros(shape=(len(home_team_player), 2), dtype=np.int8)
        home_act_nozero = [0,0] # number non-zero activity_count_all


        away_pos_count = np.zeros(shape=(len(away_team_player), 2), dtype=np.float)
        away_activity_count = np.zeros(shape=(len(away_team_player), 1), dtype=np.int8)
        away_activity_count_all = np.zeros(shape=(len(away_team_player), 2), dtype=np.int8)
        away_act_nozero = [0,0] # number non-zero activity_count_all
        
        # team positions
        home_team_role =  np.zeros(shape=(len(home_team_player), 1), dtype=np.int8)
        away_team_role =  np.zeros(shape=(len(away_team_player), 1), dtype=np.int8)


        # home and away team player count data for 10 min: 0-xpos, 1-ypos / 0-1min, 1-2min... 9-10min 
        home_pos_count_minutes = np.zeros(shape=(len(home_team_player), 2, minutes_step), dtype=np.float)
        home_activity_count_minutes = np.zeros(shape=(len(home_team_player), minutes_step), dtype=np.int8)

        home_pos_count_minutes2 = np.zeros(shape=(len(home_team_player), 2, minutes_step), dtype=np.float)
        home_activity_count_minutes2 = np.zeros(shape=(len(home_team_player), minutes_step), dtype=np.int8)

        away_pos_count_minutes = np.zeros(shape=(len(away_team_player), 2, minutes_step), dtype=np.float)
        away_activity_count_minutes = np.zeros(shape=(len(away_team_player), minutes_step), dtype=np.int8)

        away_pos_count_minutes2 = np.zeros(shape=(len(away_team_player), 2, minutes_step), dtype=np.float)
        away_activity_count_minutes = np.zeros(shape=(len(away_team_player), minutes_step), dtype=np.int8)


        # home and away team mean pos data: 0-xpos, 1-ypos
        home_pos_mean = np.zeros(shape=(len(home_team_player), 2), dtype=np.float)
        away_pos_mean = np.zeros(shape=(len(away_team_player), 2), dtype=np.float)


        # players's and refree's previous and next x&y data
        home_x_data = np.zeros(shape=(len(home_team_player),2), dtype=np.float)
        home_y_data = np.zeros(shape=(len(home_team_player),2), dtype=np.float)
        
        away_x_data = np.zeros(shape=(len(away_team_player),2), dtype=np.float)
        away_y_data = np.zeros(shape=(len(away_team_player),2), dtype=np.float)
        
        referee_xy_data = [[0,0],[0,0]]

        with open('../data/match_' + str(id) + '/per_sec_data_' + str(id) + '.json', 'r') as file:
            data_persec = json.load(file)

        home_pos_count_minutes = np.zeros(shape=(len(home_team_player), 2, minutes_step), dtype=np.float)
        home_activity_count_minutes = np.zeros(shape=(len(home_team_player), minutes_step), dtype=np.int8)

        away_pos_count_minutes = np.zeros(shape=(len(away_team_player), 2, minutes_step), dtype=np.float)
        away_activity_count_minutes = np.zeros(shape=(len(away_team_player), minutes_step), dtype=np.int8)

        # create first average positon data for assigning player position at the 1st half
        # temporary t
        
        home_activity_count_all[0: len(home_team_player)] = 0
        away_activity_count_all[0: len(away_team_player)] = 0

        
        tmp_t = 0
        tmp_t_2 = 0
        shift_times_right = 0
        
        # threshol second:
        threshold_second = 60

        # 1st half duration
        first_half_duration = 0

        home_nonzero = 11
        away_nonzero = 11

        prev_home_nonzero = 11
        prev_away_nonzero = 11
        
        
        # for detection noise data on x=0, y=0
        player_has_ball_detected = False

        # temporary half
        tmp_half = 1

        c_activity_value = 0
        c_home_activity_each = 0
        c_away_activity_each = 0

        while c_activity_value==0:

            c_home_activity_each = 0
            c_away_activity_each = 0

            a,b = fist_time_step_mean_data(threshold_second, 1, data_persec, home_pos_count_minutes,
                                        home_activity_count_minutes,away_pos_count_minutes,
                                        away_activity_count_minutes,home_dict,away_dict,
                                        home_team_player,away_team_player, home_team_id, 
                                        away_team_id, player_positionId, minutes_step)

            for player_home in range(len(home_team_player)):
                res = caluclate_activity_count(minutes_step, player_home, a)
                c_home_activity_each += res

            for player_away in range(len(away_team_player)):
                res = caluclate_activity_count(minutes_step, player_away, b)
                c_away_activity_each += res

            if c_home_activity_each == 11 and c_away_activity_each == 11:
                c_activity_value = 1

            threshold_second += 60  

        threshold_second -= 1
        tmp_t_2 = threshold_second 
        

        
        for d in data_persec:

            match_half_t = int(d['half'])

            # create first average positon data for assigning player at the 2nd half
            if match_half_t != tmp_half and match_half_t==2:
                tmp_half = match_half_t
                shift_times_right = 0

                tmp_t = 45*60

                activity_home_nonzero = 0
                activity_away_nonzero = 0
                home_break_subs = []
                away_break_subs = []
                home_break_subs_after = []
                away_break_subs_after = []

                for player_home in range(0, len(home_team_player)):
                    if home_pos_mean[player_home][0] !=0:
                            activity_home_nonzero += 1
                            home_break_subs.append(1)
                    else:
                        home_break_subs.append(0)


                for player_away in range(0, len(away_team_player)):
                    if away_pos_mean[player_away][0] !=0:
                            activity_away_nonzero += 1
                            away_break_subs.append(1)
                    else:
                        away_break_subs.append(0)

                home_nonzero = activity_home_nonzero
                away_nonzero = activity_away_nonzero


                # clear home team data
                home_pos_count[0: len(home_team_player)] = 0
                home_activity_count[0: len(home_team_player)] = 0 
                home_activity_count_all[0: len(home_team_player)] = 0
                home_act_nozero = [0,0]
                home_pos_count_minutes[0: len(home_team_player)] = 0
                home_activity_count_minutes[0: len(home_team_player)] = 0
                home_pos_mean[0: len(home_team_player)] = 0
                
                

                # clear away team data
                away_pos_count[0: len(away_team_player)] = 0
                away_activity_count[0: len(away_team_player)] = 0 
                away_activity_count_all[0: len(away_team_player)] = 0
                away_act_nozero = [0,0]
                away_pos_count_minutes[0: len(away_team_player)] = 0
                away_activity_count_minutes[0: len(away_team_player)] = 0
                away_pos_mean[0: len(away_team_player)] = 0
                
 
                # features vectors
                home_features_df[:]=0
                away_features_df[:]=0
                
                # team data
                home_df[:]=0
                away_df[:]=0
                teams_speed[:]=0
                teams_speed_data[:]=0
                
                # ball data
                ball_location[:]=0
                ball_owner[:]=0

                

                # features vectors count
                home_feature_count[:]=0
                away_feature_count[:]=0
                
                #features speed dict
                home_f_speed_dict = {0:{}, 1:{}, 2:{}, 3:{}, 4:{}, 5:{}, 6:{}}
                away_f_speed_dict = {0:{}, 1:{}, 2:{}, 3:{}, 4:{}, 5:{}, 6:{}}
                
                # index 
                ball_pos_index[0] = 0
     
                refreee_pos_noise = True

                refree_features[:]=0
                GK_home_features[:]=0
                GK_away_features[:]=0
                
                players_position_home_x[:]=0
                players_position_away_x[:]=0
                
                players_position_home_y[:]=0
                players_position_away_y[:]=0
                
                
                # previus and next x data
                home_x_data[:]=0
                away_x_data[:]=0
                referee_xy_data = [[0,0],[0,0]]

                threshold_second = 60

                c_activity_value = 0
                c_home_activity_each = 0
                c_away_activity_each = 0

                while c_activity_value==0:

                    c_home_activity_each = 0
                    c_away_activity_each = 0

                    home_break_subs_after = []
                    away_break_subs_after = []

                    a,b = fist_time_step_mean_data(threshold_second,2,data_persec,home_pos_count_minutes,
                                        home_activity_count_minutes,away_pos_count_minutes,
                                        away_activity_count_minutes,home_dict,away_dict,
                                        home_team_player,away_team_player, home_team_id, 
                                        away_team_id, player_positionId, minutes_step)

                    for player_home in range(len(home_team_player)):
                        res = caluclate_activity_count(minutes_step, player_home, a)
                        c_home_activity_each += res
                        home_break_subs_after.append(res)

                    for player_away in range(len(away_team_player)):
                        res = caluclate_activity_count(minutes_step, player_away, b)
                        c_away_activity_each += res
                        away_break_subs_after.append(res)


                    if c_home_activity_each == activity_home_nonzero and c_away_activity_each == activity_away_nonzero:
                        c_activity_value = 1

                    threshold_second += 60


                for player_home_2 in range(0, len(home_team_player)):
                    if home_break_subs_after[player_home_2]==0 and home_break_subs[player_home_2]==1:
                        home_dur_in_out[player_home_2][0] = 2 # menas goal keepr has left the game

                for player_away_2 in range(0, len(away_team_player)):
                    if away_break_subs_after[player_away_2]==0 and away_break_subs[player_away_2]==1:
                        away_dur_in_out[player_away_2][0] = 2 # menas goal keepr has left the game

                threshold_second -= 1
                tmp_t_2 = threshold_second 

            i = int(d['minute'])*60 + int(d['second'])
            x_pos = round(d['xpos'],4)
            y_pos = round(d['ypos'],4)
            team_id_t = int(d['teamId'])
            speed_t = round(d['speed'],2)
            jersey_number_t = int(d['jerseyNumber'])
            has_ball_teamId_t = int(d['hasballTeamId'])
            has_ball_jerseyId_t = int(d['hasballJerseyNumber'])
            
            if x_pos<0:
                x_pos=0.1
            elif x_pos>105:
                x_pos=105
            if y_pos<=0:
                y_pos=0.1
            elif y_pos>68:
                y_pos=68


            # find out how long did 1st half last
            if match_half_t == 1:
                first_half_duration = i


            if i%60==59:

                if team_id_t == home_team_id:
                    player = home_dict[str(jersey_number_t)]
                    home_activity_count_all[player][1] = 1

                elif team_id_t == away_team_id:
                    player = away_dict[str(jersey_number_t)]
                    away_activity_count_all[player][1] = 1
      
            if i%60!=0:    

                if team_id_t == home_team_id:

                    if home_dur_in_out[home_dict[str(jersey_number_t)]][0] == 0:
                        home_dur_in_out[home_dict[str(jersey_number_t)]][0] = 1 # indicates that a player is in game 
                        home_dur_in_out[home_dict[str(jersey_number_t)]][1] = match_half_t # enter half
                        home_dur_in_out[home_dict[str(jersey_number_t)]][2] = i  # enter time


                        plyr_position = player_positionId['homeTeam'][jersey_number_t]
                        player = home_dict[str(jersey_number_t)]
                        home_pos_count_minutes[player][1][minutes_step-1] += 34
                        home_activity_count_minutes[player][minutes_step-1] +=1

                        if match_half_t==1:
                            pos_1 = 10
                            pos_2 = 38
                            pos_3 = 58
                            pos_4 = 78
                        else:
                            pos_1 = 105-10
                            pos_2 = 105-38
                            pos_3 = 105-58
                            pos_4 = 105-78
                            

                        if plyr_position == 1:
                            home_pos_count_minutes[player][0][minutes_step-1] += pos_1
                        elif plyr_position == 2:
                            home_pos_count_minutes[player][0][minutes_step-1] += pos_2
                        elif plyr_position == 3:
                            home_pos_count_minutes[player][0][minutes_step-1] += pos_3
                        else:
                            home_pos_count_minutes[player][0][minutes_step-1] += pos_4

                    if i<((home_dur_in_out[home_dict[str(jersey_number_t)]][2])+60) and home_dur_in_out[home_dict[str(jersey_number_t)]][0] == 1:
                        if x_pos!=0 and y_pos!=0:
                            home_pos_count[home_dict[str(jersey_number_t)]][0] += x_pos
                            home_pos_count[home_dict[str(jersey_number_t)]][1] += y_pos
                            home_activity_count[home_dict[str(jersey_number_t)]] +=1 

                    elif player_positionId['homeTeam'][jersey_number_t] == 1 and has_ball_teamId_t != 0:
                        if x_pos!=0 and y_pos!=0:
                            home_pos_count[home_dict[str(jersey_number_t)]][0] += x_pos
                            home_pos_count[home_dict[str(jersey_number_t)]][1] += y_pos
                            home_activity_count[home_dict[str(jersey_number_t)]] +=1

                    elif has_ball_teamId_t == away_team_id:
                        if x_pos!=0 and y_pos!=0:
                            home_pos_count[home_dict[str(jersey_number_t)]][0] += x_pos
                            home_pos_count[home_dict[str(jersey_number_t)]][1] += y_pos
                            home_activity_count[home_dict[str(jersey_number_t)]] +=1 

                elif team_id_t == away_team_id:

                    if away_dur_in_out[away_dict[str(jersey_number_t)]][0] == 0:
                        away_dur_in_out[away_dict[str(jersey_number_t)]][0] = 1 # indicates that a player is in game
                        away_dur_in_out[away_dict[str(jersey_number_t)]][1] = match_half_t # enter half 
                        away_dur_in_out[away_dict[str(jersey_number_t)]][2] = i #enter time

                        plyr_position = player_positionId['awayTeam'][jersey_number_t]
                        player = away_dict[str(jersey_number_t)]
                        away_pos_count_minutes[player][1][minutes_step-1] += 34
                        away_activity_count_minutes[player][minutes_step-1] +=1
                        
                        
                        if match_half_t==1:
                            pos_1 = 105-10
                            pos_2 = 105-38
                            pos_3 = 105-58
                            pos_4 = 105-78
                            
                        else:
                            pos_1 = 10
                            pos_2 = 38
                            pos_3 = 58
                            pos_4 = 78
                           


                        if plyr_position == 1:
                            away_pos_count_minutes[player][0][minutes_step-1] += pos_1
                        elif plyr_position == 2:
                            away_pos_count_minutes[player][0][minutes_step-1] += pos_2
                        elif plyr_position == 3:
                            away_pos_count_minutes[player][0][minutes_step-1] += pos_3
                        else:
                            away_pos_count_minutes[player][0][minutes_step-1] += pos_4


                    if i<((away_dur_in_out[away_dict[str(jersey_number_t)]][2])+60) and away_dur_in_out[away_dict[str(jersey_number_t)]][0] == 1:
                        if x_pos!=0 and y_pos!=0:
                            away_pos_count[away_dict[str(jersey_number_t)]][0] += x_pos
                            away_pos_count[away_dict[str(jersey_number_t)]][1] += y_pos
                            away_activity_count[away_dict[str(jersey_number_t)]] +=1

                    elif player_positionId['awayTeam'][jersey_number_t] == 1 and has_ball_teamId_t != 0:
                        away_pos_count[away_dict[str(jersey_number_t)]][0] += x_pos
                        away_pos_count[away_dict[str(jersey_number_t)]][1] += y_pos
                        away_activity_count[away_dict[str(jersey_number_t)]] +=1

                    elif has_ball_teamId_t == home_team_id:
                         if x_pos!=0 and y_pos!=0:
                            away_pos_count[away_dict[str(jersey_number_t)]][0] += x_pos
                            away_pos_count[away_dict[str(jersey_number_t)]][1] += y_pos
                            away_activity_count[away_dict[str(jersey_number_t)]] +=1 

             
            # shifing last index of 15min arry (period of 15min) with last min data
            # and calculate mena position of each player each min based on previus 10 min
            if i%60 == 0 and tmp_t != i+1 and (i-(match_half_t-1)*45*60)>threshold_second:


                tmp_t = i+1

                prev_home_nonzero = home_nonzero
                home_nonzero = 0

                prev_away_nonzero = away_nonzero
                away_nonzero = 0


                if shift_times_right !=0:
                    for player_home in range(len(home_team_player)):

                        home_pos_count_minutes[player_home][0] = (shift(home_pos_count_minutes[player_home][0], -1, cval=0))
                        home_pos_count_minutes[player_home][1] = (shift(home_pos_count_minutes[player_home][1], -1, cval=0))
                        home_activity_count_minutes[player_home] = (shift(home_activity_count_minutes[player_home], -1, cval=0))

                        home_pos_count_minutes[player_home][0][minutes_step-1] = home_pos_count[player_home][0]
                        home_pos_count_minutes[player_home][1][minutes_step-1] = home_pos_count[player_home][1]
                        home_activity_count_minutes[player_home][minutes_step-1] = home_activity_count[player_home]


                    for player_away in range(len(away_team_player)):

                        away_pos_count_minutes[player_away][0] = (shift(away_pos_count_minutes[player_away][0], -1, cval=0))
                        away_pos_count_minutes[player_away][1] = (shift(away_pos_count_minutes[player_away][1], -1, cval=0))
                        away_activity_count_minutes[player_away] = (shift(away_activity_count_minutes[player_away], -1, cval=0))

                        away_pos_count_minutes[player_away][0][minutes_step-1] = away_pos_count[player_away][0]
                        away_pos_count_minutes[player_away][1][minutes_step-1] = away_pos_count[player_away][1]
                        away_activity_count_minutes[player_away][minutes_step-1] = away_activity_count[player_away]


                shift_times_right = 1


                # clear home team
                home_pos_count[0: len(home_team_player)] = 0
                home_activity_count[0: len(home_team_player)] = 0

                # clear away team
                away_pos_count[0: len(away_team_player)] = 0
                away_activity_count[0: len(away_team_player)] = 0


                home_pos_mean[0:len(home_team_player)] = 0
                home_act_nozero = [0,0]

                away_pos_mean[0:len(away_team_player)] = 0
                away_act_nozero = [0,0]

                for player_home in range(len(home_team_player)):
                    jersey_n = int(home_dict_reverse[player_home])
                    if player_positionId['homeTeam'][jersey_n] == 1 and home_dur_in_out[player_home][0]==1 and home_activity_count_all[player_home][1] == 0:
                        action_status_exit = 0
                        
                        if home_dur_in_out[player_home][4]==1:
                            enter_half = home_dur_in_out[player_home][1]
                            enter_time = home_dur_in_out[player_home][2]
                            p_game_play_dur = home_dur_in_out[player_home][3]

                            h = match_half_t
                            f_h_d= first_half_duration

                            if enter_half == 1:
                                current_play_dur = math.floor( ((f_h_d - enter_time) + (i-45*60)*(h-1) )/60 )
                            elif enter_half == 2:
                                current_play_dur = math.floor( (i - enter_time)/60 )

                            if current_play_dur < p_game_play_dur:


                                for gk_i in home_goal_keeper:
                                    if home_activity_count_all[home_dict[str(gk_i)]][1] !=0:
                                        home_dur_in_out[player_home][0] = 2 # menas goal keepr has left the game
                                        home_pos_mean[player_home][0] = 0 
                                        home_pos_mean[player_home][1] = 0
                                        action_status_exit = 1
                        else:
                            for gk_i in home_goal_keeper:
                                if home_activity_count_all[home_dict[str(gk_i)]][1] !=0:
                                    home_dur_in_out[player_home][0] = 2 # menas goal keepr has left the game
                                    home_pos_mean[player_home][0] = 0 
                                    home_pos_mean[player_home][1] = 0
                                    action_status_exit = 1
                            

                        if action_status_exit == 0:
                            home_activity_count_all[player_home][1] = 1

                    if (home_activity_count_all[player_home][0]==1):
                        home_act_nozero[0] += 1

                    if (home_activity_count_all[player_home][1]==1):
                        home_act_nozero[1] += 1
                        
                

                for player_away in range(len(away_team_player)):
                    jersey_n = int(away_dict_reverse[player_away])
                    if player_positionId['awayTeam'][jersey_n] == 1 and away_dur_in_out[player_away][0]==1 and away_activity_count_all[player_away][1] == 0:
                        action_status_exit = 0
                        
                        if away_dur_in_out[player_away][4]==1:

                            # player current and whole game duration durng match
                            enter_half = away_dur_in_out[player_away][1]
                            enter_time = away_dur_in_out[player_away][2]
                            p_game_play_dur = away_dur_in_out[player_away][3]

                            h = match_half_t
                            f_h_d= first_half_duration

                            if enter_half == 1:
                                current_play_dur = math.floor( ((f_h_d - enter_time) + (i-45*60)*(h-1) )/60 )
                            elif enter_half == 2:
                                current_play_dur = math.floor( (i - enter_time)/60 )

                            if current_play_dur < p_game_play_dur:

                                for gk_i in away_goal_keeper:
                                    if away_activity_count_all[away_dict[str(gk_i)]][1] !=0:
                                        away_dur_in_out[player_away][0] = 2 # menas goal keepr has left the game
                                        away_pos_mean[player_away][0] = 0 
                                        away_pos_mean[player_away][1] = 0
                                        action_status_exit = 1
                        else:
                            for gk_i in away_goal_keeper:
                                if away_activity_count_all[away_dict[str(gk_i)]][1] !=0:
                                    away_dur_in_out[player_away][0] = 2 # menas goal keepr has left the game
                                    away_pos_mean[player_away][0] = 0 
                                    away_pos_mean[player_away][1] = 0
                                    action_status_exit = 1
                            
                        if action_status_exit == 0:
                            away_activity_count_all[player_away][1] = 1


                    if (away_activity_count_all[player_away][0]!=0):
                        away_act_nozero[0] += 1

                    if (away_activity_count_all[player_away][1]!=0):
                        away_act_nozero[1] += 1

                home_subs_count_check = 0
                home_subs_count_id = 0
                home_2subs_array_in = []
                home_2subs_array_out = []


                for player_home in range(len(home_team_player)):
                  

                    if home_activity_count_all[player_home][1] != 0 and home_dur_in_out[player_home][0]==1:

                        caluclate_avrg_pos(minutes_step, home_pos_mean, player_home, home_pos_count_minutes, 
                                           home_activity_count_minutes)

                        if home_act_nozero[0]-home_act_nozero[1]<0 and home_act_nozero[0]!=0 and home_activity_count_all[player_home][0] == 0:
                            home_pos_mean[player_home][0] = 0 
                            home_pos_mean[player_home][1] = 0
                            home_activity_count_all[player_home][1] = 0


                    elif home_activity_count_all[player_home][1] == 0 and home_dur_in_out[player_home][0]==1 and home_dur_in_out[player_home][4]==1: 
                        # player current and whole game duration durng match


                        enter_half = home_dur_in_out[player_home][1]
                        enter_time = home_dur_in_out[player_home][2]
                        p_game_play_dur = home_dur_in_out[player_home][3]

                        h = match_half_t
                        f_h_d= first_half_duration

                        if enter_half == 1:
                            current_play_dur = math.floor( ((f_h_d - enter_time) + (i-45*60)*(h-1) )/60 )
                        elif enter_half == 2:
                            current_play_dur = math.floor( (i - enter_time)/60 )

                        if current_play_dur < p_game_play_dur:
                            caluclate_avrg_pos(minutes_step, home_pos_mean, player_home, home_pos_count_minutes, 
                                           home_activity_count_minutes)
                            home_activity_count_all[player_home][1] = 1
                        else:
                            home_dur_in_out[player_home][0] = 2
                            home_pos_mean[player_home][0] = 0 
                            home_pos_mean[player_home][1] = 0


                    elif home_activity_count_all[player_home][1] == 0 and home_dur_in_out[player_home][0]==1:

                        if (home_act_nozero[0] == home_act_nozero[1]):
                            home_dur_in_out[player_home][0]=2
                            home_pos_mean[player_home][0] = 0 
                            home_pos_mean[player_home][1] = 0

                        elif home_act_nozero[0] > home_act_nozero[1]:
                            caluclate_avrg_pos(minutes_step, home_pos_mean, player_home, home_pos_count_minutes, 
                                           home_activity_count_minutes)
                            home_activity_count_all[player_home][1] = 1
                            home_subs_count_check += 1
                            home_2subs_array_out.append(player_home)
                            home_subs_count_id = player_home

                    else:

                        home_pos_mean[player_home][0] = 0 
                        home_pos_mean[player_home][1] = 0


                for player_home in range(len(home_team_player)):
                    if home_subs_count_check >=1 and (home_act_nozero[0]-home_act_nozero[1]==1):

                        d = caluclate_nzero_activity_count(minutes_step, 0, home_subs_count_id, home_activity_count_minutes)
                        if home_activity_count_all[player_home][0] == 0 and home_activity_count_all[player_home][1] == 1:
                            
                            if d==14:
                                d-=2
                            result = caluclate_nzero_activity_count(minutes_step, d+1, player_home, home_activity_count_minutes)
                            if result != -1:
                                home_2subs_array_in.append(player_home)

                    home_activity_count_all[player_home][0] = home_activity_count_all[player_home][1]
                    home_activity_count_all[player_home][1] = 0



                # number of last non-zero activties
                home_last_nz_act_count = caluclate_last_nzero_ac_count(len(home_team_player), home_activity_count_all)



                if home_last_nz_act_count == prev_home_nonzero and len(home_2subs_array_out)==2:
                    for home_2subs_i in range(0,2):
                        player = home_2subs_array_out[home_2subs_i]
                        if home_activity_count_minutes[player][minutes_step-1] == 0:
                            home_dur_in_out[player][0] = 2
                            home_activity_count_all[player][0] = 0
                            home_pos_mean[player][0] = 0
                            home_pos_mean[player][1] = 0


                if len(home_2subs_array_out)>=2:
                    for home_2subs_i in range(0,2):
                        player = home_2subs_array_out[home_2subs_i]
                        home_activity_count_all[player][0] = 1
                        caluclate_avrg_pos(minutes_step, home_pos_mean, player_home, home_pos_count_minutes, 
                                           home_activity_count_minutes)


                if len(home_2subs_array_in) == 2 and len(home_2subs_array_out)==2:
                    for home_2subs_i in range(0,2):
                        player = home_2subs_array_out[home_2subs_i]
                        home_dur_in_out[player][0] = 2
                        home_activity_count_all[player][0] = 0
                        home_pos_mean[player][0] = 0
                        home_pos_mean[player][1] = 0

                if len(home_2subs_array_in) == 1 and len(home_2subs_array_out)>=2:
                    player = home_2subs_array_in[0]
                    home_activity_count_all[player][0] = 0
                    home_pos_mean[player][0] = 0
                    home_pos_mean[player][1] = 0


                if home_act_nozero[0]-home_act_nozero[1]==1 and home_subs_count_check==1 and len(home_2subs_array_in) == 1:
                    player_out = home_2subs_array_out[0]
                    home_dur_in_out[player_out][0] = 2
                    home_activity_count_all[player_out][0] = 0
                    home_pos_mean[player_out][0] = 0
                    home_pos_mean[player_out][1] = 0
                    

                for player_home in range(len(home_team_player)):
                    x = home_pos_mean[player_home][0]
                    y = home_pos_mean[player_home][1]
                    
                    if x!=0:
                        home_nonzero += 1
                        
                if home_nonzero != home_prev_count:
                    break
                
                # scaling home position
                temporary_pos_home = np.zeros(shape=(10,2), dtype=np.float)
                tmp_pos_i = 0
                for player_home in range(len(home_team_player)):
                    st_jersey_n = int(home_dict_reverse[player_home])
                    if home_pos_mean[player_home][0]!=0 and player_positionId['homeTeam'][st_jersey_n] != 1:
                        temporary_pos_home[tmp_pos_i][0] = home_pos_mean[player_home][0]
                        temporary_pos_home[tmp_pos_i][1] = home_pos_mean[player_home][1]
                        tmp_pos_i +=1
                        
                        
                STDx = np.std(temporary_pos_home[:,0])
                STDy = np.std(temporary_pos_home[:,1])

                temporary_pos_home[:,0] = scale_linear_data(temporary_pos_home[:,0], 105-STDx, STDx)
                temporary_pos_home[:,1] = scale_linear_data(temporary_pos_home[:,1], 68-STDy, STDy)
                
                tmp_pos_i = 0
                for player_home in range(len(home_team_player)):
                    st_jersey_n = int(home_dict_reverse[player_home])
                    if home_pos_mean[player_home][0]!=0 and player_positionId['homeTeam'][st_jersey_n] != 1:
                        home_pos_mean[player_home][0] = temporary_pos_home[tmp_pos_i][0]
                        home_pos_mean[player_home][1] = temporary_pos_home[tmp_pos_i][1]
                        tmp_pos_i +=1
                        

                for player_home in range(len(home_team_player)):
                    x = home_pos_mean[player_home][0]
                    y = home_pos_mean[player_home][1]
                    gk_state=0
                    
                    st_jersey_n = int(home_dict_reverse[player_home])
                    if player_positionId['homeTeam'][st_jersey_n] == 1:
                        gk_state=1
            
                    if x!=0:
                        home_team_role[player_home] = define_role(x, y, role_segment_coord, gk_state)
                    else:
                        home_team_role[player_home] = -1

                        
                away_subs_count_check = 0
                away_subs_count_id = 0
                away_2subs_array_in = []
                away_2subs_array_out = []

                for player_away in range(len(away_team_player)):


                    if away_activity_count_all[player_away][1] != 0 and away_dur_in_out[player_away][0]==1:

                        caluclate_avrg_pos(minutes_step, away_pos_mean, player_away, away_pos_count_minutes, 
                                               away_activity_count_minutes)

                        if away_act_nozero[0]-away_act_nozero[1]<0 and away_act_nozero[0]!=0 and away_activity_count_all[player_away][0] == 0:
                            away_pos_mean[player_away][0] = 0 
                            away_pos_mean[player_away][1] = 0
                            away_activity_count_all[player_away][1] = 0

                    elif away_activity_count_all[player_away][1] == 0 and away_dur_in_out[player_away][0]==1 and away_dur_in_out[player_away][4]==1:


                        # player current and whole game duration durng match
                        enter_half = away_dur_in_out[player_away][1]
                        enter_time = away_dur_in_out[player_away][2]
                        p_game_play_dur = away_dur_in_out[player_away][3]

                        h = match_half_t
                        f_h_d= first_half_duration

                        if enter_half == 1:
                            current_play_dur = math.floor( ((f_h_d - enter_time) + (i-45*60)*(h-1) )/60 )
                        elif enter_half == 2:
                            current_play_dur = math.floor( (i - enter_time)/60 )

                        if current_play_dur < p_game_play_dur:
                            caluclate_avrg_pos(minutes_step, away_pos_mean, player_away, away_pos_count_minutes, 
                                           away_activity_count_minutes)
                            away_activity_count_all[player_away][1] = 1

                        else:
                            away_dur_in_out[player_away][0] = 2
                            away_pos_mean[player_away][0] = 0 
                            away_pos_mean[player_away][1] = 0


                    elif away_activity_count_all[player_away][1] == 0 and away_dur_in_out[player_away][0]==1:

                        if away_act_nozero[0] == away_act_nozero[1]:
                            away_dur_in_out[player_away][0] = 2
                            away_pos_mean[player_away][0] = 0 
                            away_pos_mean[player_away][1] = 0


                        elif away_act_nozero[0] > away_act_nozero[1]:
                            caluclate_avrg_pos(minutes_step, away_pos_mean, player_away, away_pos_count_minutes, 
                                           away_activity_count_minutes)
                            away_activity_count_all[player_away][1] = 1
                            away_subs_count_check += 1
                            away_2subs_array_out.append(player_away)
                            away_subs_count_id = player_away

                    else:
                        away_pos_mean[player_away][0] = 0 
                        away_pos_mean[player_away][1] = 0


                for player_away in range(len(away_team_player)):
                    if away_subs_count_check >= 1 and (away_act_nozero[0]-away_act_nozero[1]==1):
                            d = caluclate_nzero_activity_count(minutes_step, 0, away_subs_count_id, away_activity_count_minutes)
                            if away_activity_count_all[player_away][0] ==0 and away_activity_count_all[player_away][1] == 1:
                                if d==14:
                                    d-=2
                                result = caluclate_nzero_activity_count(minutes_step, d+1, player_away, away_activity_count_minutes)
                                if result != -1:
                                    away_2subs_array_in.append(player_away)

                    away_activity_count_all[player_away][0] = away_activity_count_all[player_away][1]
                    away_activity_count_all[player_away][1] = 0


                # number of last non-zero activties
                away_last_nz_act_count = caluclate_last_nzero_ac_count(len(away_team_player), away_activity_count_all)

                if away_last_nz_act_count == prev_away_nonzero and len(away_2subs_array_out)==2:
                    for away_2subs_i in range(0,2):
                        player = away_2subs_array_out[away_2subs_i]
                        if away_activity_count_minutes[player][minutes_step-1] == 0:
                            away_dur_in_out[player][0] = 2
                            away_activity_count_all[player][0] = 0
                            away_pos_mean[player][0] = 0
                            away_pos_mean[player][1] = 0


                if len(away_2subs_array_out)>=2:
                    for away_2subs_i in range(0,2):
                        player = away_2subs_array_out[away_2subs_i]
                        away_activity_count_all[player][0] = 1

                if len(away_2subs_array_in) == 2 and away_subs_count_check==2:
                    for away_2subs_i in range(0,2):
                        player = away_2subs_array_out[away_2subs_i]
                        away_dur_in_out[player][0] = 2
                        away_activity_count_all[player][0] = 0
                        away_pos_mean[player][0] = 0
                        away_pos_mean[player][1] = 0

                if len(away_2subs_array_in) == 1 and away_subs_count_check>=2:
                    player = away_2subs_array_in[0]
                    away_activity_count_all[player][0] = 0
                    away_pos_mean[player][0] = 0
                    away_pos_mean[player][1] = 0

                if away_act_nozero[0]-away_act_nozero[1]==1 and away_subs_count_check==1 and len(away_2subs_array_in) == 1:
                    player_out = away_2subs_array_out[0]
                    away_dur_in_out[player_out][0] = 2
                    away_activity_count_all[player_out][0] = 0
                    away_pos_mean[player_out][0] = 0
                    away_pos_mean[player_out][1] = 0

                
                for player_away in range(len(away_team_player)):
                    x = away_pos_mean[player_away][0]
                    y = away_pos_mean[player_away][1]
                    
                    if x!=0:
                        away_nonzero += 1
                        
                if away_nonzero != away_prev_count:
                    break

                # converting away X position and scaling them

                temporary_pos_away = np.zeros(shape=(10,2), dtype=np.float)
                
                tmp_pos_i = 0
                for player_away in range(len(away_team_player)):
                    st_jersey_n = int(away_dict_reverse[player_away])
                    
                    if away_pos_mean[player_away][0]!=0:
                        away_pos_mean[player_away][0] = 105-away_pos_mean[player_away][0]
                        if player_positionId['awayTeam'][st_jersey_n] != 1:
                            temporary_pos_away[tmp_pos_i][0] = away_pos_mean[player_away][0]
                            temporary_pos_away[tmp_pos_i][1] = away_pos_mean[player_away][1]
                            tmp_pos_i +=1
                            
                
                STDx = np.std(temporary_pos_away[:,0])
                STDy = np.std(temporary_pos_away[:,1])

                temporary_pos_away[:,0] = scale_linear_data(temporary_pos_away[:,0], 105-STDx, STDx)
                temporary_pos_away[:,1] = scale_linear_data(temporary_pos_away[:,1], 68-STDy, STDy)
                
                tmp_pos_i = 0
                for player_away in range(len(away_team_player)):
                    st_jersey_n = int(away_dict_reverse[player_away])
                    if away_pos_mean[player_away][0]!=0 and player_positionId['awayTeam'][st_jersey_n] != 1:
                        away_pos_mean[player_away][0] = temporary_pos_away[tmp_pos_i][0]
                        away_pos_mean[player_away][1] = temporary_pos_away[tmp_pos_i][1]
                        tmp_pos_i +=1
                

                for player_away in range(0, len(away_team_player)):
                    x = away_pos_mean[player_away][0]
                    y = away_pos_mean[player_away][1]
                    
                    gk_state=0
                    
                    st_jersey_n = int(away_dict_reverse[player_away])
                    if player_positionId['awayTeam'][st_jersey_n] == 1:
                        gk_state=1
            
                    if x!=0:
                        away_team_role[player_away] = define_role(x, y, role_segment_coord, gk_state)
                    else:
                        away_team_role[player_away] = -1

                    
            elif (i-(match_half_t-1)*45*60)==threshold_second:
                
                for player_home in range(len(home_team_player)):
                    caluclate_avrg_pos(minutes_step, home_pos_mean, player_home, home_pos_count_minutes, 
                                           home_activity_count_minutes)
                    
                for player_away in range(len(away_team_player)):
                    caluclate_avrg_pos(minutes_step, away_pos_mean, player_away, away_pos_count_minutes, 
                                               away_activity_count_minutes)
                    
                    
                # converting away X position and scaling them

                temporary_pos_away = np.zeros(shape=(10,2), dtype=np.float)
                
                tmp_pos_i = 0
                for player_away in range(len(away_team_player)):
                    st_jersey_n = int(away_dict_reverse[player_away])
                    
                    if away_pos_mean[player_away][0]!=0:
                        away_pos_mean[player_away][0] = 105-away_pos_mean[player_away][0]
                        if player_positionId['awayTeam'][st_jersey_n] != 1:
                            temporary_pos_away[tmp_pos_i][0] = away_pos_mean[player_away][0]
                            temporary_pos_away[tmp_pos_i][1] = away_pos_mean[player_away][1]
                            tmp_pos_i +=1
                        

                STDx = np.std(temporary_pos_away[:,0])
                STDy = np.std(temporary_pos_away[:,1])        
                        
                temporary_pos_away[:,0] = scale_linear_data(temporary_pos_away[:,0], 105-STDx, STDx)
                temporary_pos_away[:,1] = scale_linear_data(temporary_pos_away[:,1], 68-STDx, STDy)
                
                tmp_pos_i = 0
                for player_away in range(0, len(away_team_player)):
                    st_jersey_n = int(away_dict_reverse[player_away])
                    if away_pos_mean[player_away][0]!=0 and player_positionId['awayTeam'][st_jersey_n] != 1:
                        away_pos_mean[player_away][0] = temporary_pos_away[tmp_pos_i][0]
                        away_pos_mean[player_away][1] = temporary_pos_away[tmp_pos_i][1]
                        tmp_pos_i +=1
                
                
                # scaling home position
                temporary_pos_home = np.zeros(shape=(10,2), dtype=np.float)
                tmp_pos_i = 0
                for player_home in range(0, len(home_team_player)):
                    st_jersey_n = int(home_dict_reverse[player_home])
                    if home_pos_mean[player_home][0]!=0 and player_positionId['homeTeam'][st_jersey_n] != 1:
                        temporary_pos_home[tmp_pos_i][0] = home_pos_mean[player_home][0]
                        temporary_pos_home[tmp_pos_i][1] = home_pos_mean[player_home][1]
                        tmp_pos_i +=1
                        
                    
                STDx = np.std(temporary_pos_home[:,0])
                STDy = np.std(temporary_pos_home[:,1]) 
                
                temporary_pos_home[:,0] = scale_linear_data(temporary_pos_home[:,0], 105-STDx, STDx)
                temporary_pos_home[:,1] = scale_linear_data(temporary_pos_home[:,1], 68-STDx, STDy)
                
                tmp_pos_i = 0
                for player_home in range(len(home_team_player)):
                    st_jersey_n = int(home_dict_reverse[player_home])
                    if home_pos_mean[player_home][0]!=0 and player_positionId['homeTeam'][st_jersey_n] != 1:
                        home_pos_mean[player_home][0] = temporary_pos_home[tmp_pos_i][0]
                        home_pos_mean[player_home][1] = temporary_pos_home[tmp_pos_i][1]
                        tmp_pos_i +=1
                
  
                for player_home in range(len(home_team_player)):
                     
                    x = home_pos_mean[player_home][0]
                    y = home_pos_mean[player_home][1]
                    
                    gk_state=0
                    
                    st_jersey_n = int(home_dict_reverse[player_home])
                    if player_positionId['homeTeam'][st_jersey_n] == 1:
                        gk_state=1
            
                    if x!=0:
                        home_team_role[player_home] = define_role(x, y, role_segment_coord,gk_state)
                    else:
                        home_team_role[player_home] = -1
                    
                
                for player_away in range(len(away_team_player)):
                    
                    x = away_pos_mean[player_away][0]
                    y = away_pos_mean[player_away][1]
                    
                    gk_state=0
                    
                    st_jersey_n = int(away_dict_reverse[player_away])
                    if player_positionId['awayTeam'][st_jersey_n] == 1:
                        gk_state=1

                        
                    if x!=0:
                        away_team_role[player_away] = define_role(x, y, role_segment_coord, gk_state)
                    else:
                        away_team_role[player_away] = -1

                        
            if (i-(match_half_t-1)*45*60)>=threshold_second:
                
                if tmp_t_2!=i:
                       
                    tmp_t_2=i
                    
                    home_count=0
                    for h_data_i in home_x_data:
                        if h_data_i[1]!=0:
                            home_count+=1
                            
                    away_count=0
                    for a_data_i in away_x_data:
                        if a_data_i[1]!=0:
                            away_count+=1
                    
                    if home_count>4 and away_count>4:# and refreee_pos_noise == False: 
                     
                        teamDBSCAN = np.zeros(shape=(2), dtype=np.float)
                        homeDBSCAN = np.zeros(shape=(2), dtype=np.float)
                        awayDBSCAN = np.zeros(shape=(2), dtype=np.float)

                        
                        
                        hx_data_tmp =[]
                        hy_data_tmp =[]
                        ax_data_tmp =[]
                        ay_data_tmp =[]

                   
                        players_position_home_x[0] = GK_home_features[0][0]
                        players_position_home_y[0] = GK_home_features[0][1] 
                        
                        count_x=1
                        count_y=1
                        for p_i in range(len(home_x_data)):
                            if home_x_data[p_i,1]!=0:
                                hx_data_tmp.append(home_x_data[p_i,1])
                                hy_data_tmp.append(home_y_data[p_i,1])
                            st_jersey_n = int(home_dict_reverse[p_i])
                            if player_positionId['homeTeam'][st_jersey_n] != 1:
                                players_position_home_x[count_x] = home_x_data[p_i,1]
                                players_position_home_y[count_y] = home_y_data[p_i,1]
                                count_y+=1
                                count_x+=1
                        
                        players_position_away_x[0] = GK_away_features[0][0]
                        players_position_away_y[0] = GK_away_features[0][1]   

                        count_x=1
                        count_y=1
                        for p_i in range(len(away_x_data)):
                            if away_x_data[p_i,1]!=0:
                                ax_data_tmp.append(away_x_data[p_i,1])
                                ay_data_tmp.append(away_y_data[p_i,1])
                            st_jersey_n = int(away_dict_reverse[p_i])
                            if player_positionId['awayTeam'][st_jersey_n] != 1:
                                players_position_away_x[count_x] = away_x_data[p_i,1]
                                players_position_away_y[count_y] = away_y_data[p_i,1]
                                count_y+=1
                                count_x+=1
                                
                       
                                
            
                        
                        all_pos_temp = np.concatenate(( match_info,          # match id, half, min, sec
                                                        players_position_home_x,
                                                        players_position_home_y,
                                                        players_position_away_x,
                                                        players_position_away_y
                                                      ), axis=0)
                        all_pos_temp = np.resize(all_pos_temp,(1,len(all_pos_temp)))
                        
                        all_pos_data.append(all_pos_temp)

                        x_all = np.array([np.concatenate((hx_data_tmp,ax_data_tmp), axis=0), np.concatenate((hy_data_tmp, ay_data_tmp), axis=0)])
                        x_all = np.transpose(x_all)
                        # homw team x-based data
                        x_home = x_all[0:len(hx_data_tmp)]
                        # away team x-based data
                        x_away = x_all[len(hx_data_tmp):-1]

                        # all
                        c_n, _ = x_all.shape
                        X_all = np.zeros(shape=(c_n, 2), dtype=np.float)
                        for r_i in range(c_n):
                             X_all[r_i][0] = x_all[r_i][0]

                        # home
                        c_n, _ = x_home.shape
                        X_home = np.zeros(shape=(c_n, 2), dtype=np.float)
                        for r_i in range(c_n):
                            X_home[r_i][0] = x_home[r_i][0]
 
                        # away
                        c_n, _ = x_away.shape
                        X_away = np.zeros(shape=(c_n, 2), dtype=np.float)
                        for r_i in range(c_n):
                            X_away[r_i][0] = x_away[r_i][0]
                            
                            
                        # teamDBSCAN
                        teamDBSCAN =  innerDBSCAN(X_all, x_all, 10, 10)
                        # homeDBSCAN
                        homeDBSCAN =  innerDBSCAN(X_home, x_home, 9, 6)
                        # awayDBSCAN
                        awayDBSCAN =  innerDBSCAN(X_away, x_away, 9, 6)
                        
                        teams_speed[0]=0

                        features_adjust(home_features_df, home_df, home_f_speed_dict, home_x_data, home_y_data,teams_speed)
                        features_adjust(away_features_df, away_df, away_f_speed_dict, away_x_data, away_y_data,teams_speed)
                        
                        teams_speed_data[0]= min(home_df[9], away_df[9])
                        teams_speed_data[1]= min(home_df[12], away_df[12])
                        teams_speed_data[2]= min(home_df[15], away_df[15])
                        
                        teams_speed[0] /= (home_count+away_count)

                        refree_fd = np.resize(refree_features,(5))
                        GK_home_fd = np.resize(GK_home_features,(3))
                        GK_away_fd = np.resize(GK_away_features,(3))

                        
                        match_info[1]=match_half_t
                        match_info[2]=i//60
                        match_info[3]=i%60
                        
                        all_data_temp = np.concatenate((match_info,          # match id, half, min, sec
                            
                                                        refree_fd,           # referee features
                                                        
                                                        teams_speed,         # teamsAvgSpeed
                                                        teams_speed_data,    # teamsMinMaxSprintSpeed
                                                        teamDBSCAN,          # teamDBSCAN x,y

                                                        homeDBSCAN,          # homeDBSCAN x,y
                                                        home_df,             # homeTeam features
                                                        GK_home_fd,          # goalKepper features
                                                        home_features_df[0], # homeLeftBack features
                                                        home_features_df[1], # homeCenterBack features
                                                        home_features_df[2], # homeRightBack features
                                                        home_features_df[3], # homeLeftWinger features
                                                        home_features_df[4], # homeCenterMiddle features
                                                        home_features_df[5], # homeRightWinger features
                                                        home_features_df[6], # homeCenterForward features

                                                        awayDBSCAN,          # awayDBSCAN x,y
                                                        away_df,             # awayTeam features
                                                        GK_away_fd,          # goalKepper features
                                                        away_features_df[0], # awayLeftBack features
                                                        away_features_df[1], # awayCenterBack features
                                                        away_features_df[2], # awayRightBack features
                                                        away_features_df[3], # awayLeftWinger features
                                                        away_features_df[4], # awayCenterMiddle features
                                                        away_features_df[5], # awayRightWinger features
                                                        away_features_df[6], # awayCenterForward features

                                                        ball_pos_index,
                                                        ball_location,
                                                        ball_owner,
                                                        
                                                       ), axis=0)

                        all_data_temp = np.resize(all_data_temp,(1,len(all_data_temp)))



#                         additional_data = np.array([id, match_half_t, i//60, i%60])

#                         features_count_tmp = np.concatenate((additional_data, 
#                                                       home_feature_count[0], # homeLeftBack counts
#                                                       home_feature_count[1], # homeCenterBack counts
#                                                       home_feature_count[2], # homeRightBack counts
#                                                       home_feature_count[3], # homeLeftWinger counts
#                                                       home_feature_count[4], # homeCenterMiddle counts
#                                                       home_feature_count[5], # homeRightWinger counts
#                                                       home_feature_count[6], # homeCenterForward counts

#                                                       away_feature_count[0], # awayLeftBack counts
#                                                       away_feature_count[1], # awayCenterBack counts
#                                                       away_feature_count[2], # awayRightBack counts
#                                                       away_feature_count[3], # awayLeftWinger counts
#                                                       away_feature_count[4], # awayCenterMiddle counts
#                                                       away_feature_count[5], # awayRightWinger counts
#                                                       away_feature_count[6], # awayCenterForward counts
#                                                       ), axis=0)

#                         features_count_tmp = np.resize(features_count_tmp,(1,len(features_count_tmp)))

#                         all_data_count.append(features_count_tmp)
                        all_data.append(all_data_temp)
    
                        

                    # then clear every thing
                    home_features_df[:] = 0
                    away_features_df[:] = 0
                    
                    home_df[:]=0
                    away_df[:]=0
                    
                    ball_location[:]=0
                    ball_owner[:]=0
                    
                    players_position_home_x[:]=0
                    players_position_away_x[:]=0
                    
                    players_position_home_y[:]=0
                    players_position_away_y[:]=0
                    
                    teams_speed[:]=0
                    teams_speed_data[:]=0

                    
                    home_feature_count[:] = 0
                    away_feature_count[:] = 0
                    
                    
                    refree_features[:] = 0
                    GK_home_features[:] = 0
                    GK_away_features[:] = 0
                    ball_pos_index[0] = 0
                    
                    refreee_pos_noise = True
                    
                    
                    
                    home_f_speed_dict = {0:{}, 1:{}, 2:{}, 3:{}, 4:{}, 5:{}, 6:{}}
                    away_f_speed_dict = {0:{}, 1:{}, 2:{}, 3:{}, 4:{}, 5:{}, 6:{}}
                    
                    
                    for i in range(0, len(home_team_player)):
                        home_x_data[i][0]=home_x_data[i][1]
                        home_x_data[i][1]=0
                        home_y_data[i][0]=home_y_data[i][1]
                        home_y_data[i][1]=0
                    
                    for i in range(0, len(away_team_player)):
                        away_x_data[i][0]=away_x_data[i][1]
                        away_x_data[i][1]=0
                        away_y_data[i][0]=away_y_data[i][1]
                        away_y_data[i][1]=0
                    
                        
                    referee_xy_data[0][0] = referee_xy_data[0][1]
                    referee_xy_data[1][0] = referee_xy_data[1][1]
                        
                
                # REFREEE:
                if team_id_t==0 and jersey_number_t==0:
                    if x_pos!=0 or y_pos!=0:
                    
                        refreee_pos_noise = False

                        refree_features[0][0] = x_pos
                        refree_features[0][1] = y_pos
                        refree_features[0][2] = speed_t

                        referee_xy_data[0][1] = x_pos
                        referee_xy_data[1][1] = y_pos

                        deltaX = x_pos-referee_xy_data[0][0]
                        deltaY = y_pos-referee_xy_data[1][0]

                        refree_features[0][3] = define_direction(deltaX)
                        refree_features[0][4] = define_direction(deltaY)
                    


                # HOME
                elif team_id_t == home_team_id: 
                    h_jersey_no = home_dict[str(jersey_number_t)]
                    r = int(home_team_role[h_jersey_no])
                    
                    if has_ball_teamId_t == home_team_id and has_ball_jerseyId_t == jersey_number_t:
                        if x_pos!=0 or y_pos!=0:
                            ball_pos_index[0] = define_pitch_index(x_pos,y_pos)
                            ball_owner[0]=1
                            ball_location[0] = x_pos
                            ball_location[1] = y_pos
                       

                    # Goalkepper
                    if r==7:
                        
                        GK_home_features[0][0] = x_pos
                        GK_home_features[0][1] = y_pos
                        GK_home_features[0][2] = speed_t

                    #PLAYERS
                    elif r!=-1:
                        if x_pos!=0 or y_pos!=0:
                            home_x_data[h_jersey_no][1] = x_pos
                            home_y_data[h_jersey_no][1] = y_pos
                            feature_generation(r,speed_t,h_jersey_no,home_feature_count, home_f_speed_dict)
                        
                # AWAY        
                elif team_id_t == away_team_id: 
                    a_jersey_no = away_dict[str(jersey_number_t)]
                    r = int(away_team_role[a_jersey_no])
                    
                    if has_ball_teamId_t == away_team_id and has_ball_jerseyId_t== jersey_number_t:
                        if x_pos!=0 or y_pos!=0:
                            ball_pos_index[0] = define_pitch_index(x_pos,y_pos)
                            ball_owner[0]=1
                            ball_location[0] = x_pos
                            ball_location[1] = y_pos
                        
                    # Goalkepper
                    if r==7:
                        GK_away_features[0][0] = x_pos
                        GK_away_features[0][1] = y_pos
                        GK_away_features[0][2] = speed_t
                        
                    # PLAYERS
                    elif r!=-1:
                        if x_pos!=0 or y_pos!=0:
                            away_x_data[a_jersey_no][1] = x_pos
                            away_y_data[a_jersey_no][1] = y_pos
                            feature_generation(r,speed_t,a_jersey_no,away_feature_count, away_f_speed_dict)
    
        
#         #writing players' role assignments count to csv file
#         all_data_count = np.array(all_data_count, dtype=np.int)
#         length_s_count, _, width_s_count = all_data_count.shape
#         all_data_count = np.resize(all_data_count,(length_s_count,width_s_count))
#         all_data_count = all_data_count.astype(int)

        all_data = np.array(all_data, dtype=np.float16)
        length_f, _, width_f = all_data.shape
        all_data = np.resize(all_data,(length_f,width_f))
        
       
        
        all_pos_data = np.array(all_pos_data, dtype=np.float16)
        length_f, _, width_f = all_pos_data.shape
        all_pos_data = np.resize(all_pos_data,(length_f,width_f))
        
        np.savetxt('../data/general/feature-set/players_pos_{}.csv'.format(id), all_pos_data, delimiter=',', header=pos_header)
        
        np.savetxt('../data/general/feature-set/match{}_features.csv'.format(id), all_data, delimiter=',', header = features_set_header)
#         np.savetxt('../data/match_{}/role_assignment_count_v2.csv'.format(id), all_data_count, delimiter=',', header = group_count_header)

#         pd.options.display.float_format = '{:.0f}'.format 
#         roleassign_df = pd.read_csv('../data/match_{}/role_assignment_count_v2.csv'.format(id))
#         display(roleassign_df.tail())
        
        pd.options.display.float_format = '{:.5f}'.format 
        features_df = pd.read_csv('../data/general/feature-set/match{}_features.csv'.format(id))
        player_pos_df = pd.read_csv('../data/general/feature-set/players_pos_{}.csv'.format(id))

        display(features_df.tail())
        display(player_pos_df.tail())


Match id:60565


,# match_id,half,min,second,refX,refY,refSpeed,refDirectX,refDirectY,teamsAvgSpeed,teamsMinSpeed,teamMaxSpeed,teamsMaxSprint,teamsDBSCANx,teamsDBSCANy,homeDBSCANx,homeDBSCANy,homeMaxX,homeMinX,homeMaxY,homeMinY,homeAvgX,homeAvgY,homeAvgSpeed,homeMinSpeedX,homeMinSpeedY,homeMinSpeed,homeMaxSpeedX,homeMaxSpeedY,homeMaxSpeed,homeMaxSprintX,homeMaxSprintY,homeMaxSprint,homeGoalKeeperX,homeGoalKeeperY,homeGoalKeeperSpeed,homeLeftBackMaxX,homeLeftBackMinX,homeLeftBackMaxY,homeLeftBackMinY,homeLeftBackAvgX,homeLeftBackAvgY,homeLeftBackAvgSpeed,homeLeftBackSlowAvgX,homeLeftBackSlowAvgY,homeLeftBackHirAvgX,homeLeftBackHirAvgY,homeLeftBackMaxSprintX,homeLeftBackMaxSprintY,homeLeftBackMaxSprintSpeed,homeLeftBackMinSpeedX,homeLeftBackMinSpeedY,homeLeftBackMinSpeed,homeLeftBackMaxSpeedX,homeLeftBackMaxSpeedY,homeLeftBackMaxSpeed,homeCenterBackMaxX,homeCenterBackMinX,homeCenterBackMaxY,homeCenterBackMinY,homeCenterBackAvgX,homeCenterBackAvgY,homeCenterBackAvgSpeed,homeCenterBackSlowAvgX,homeCenterBackSlowAvgY,homeCenterBackHirAvgX,homeCenterBackHirAvgY,homeCenterBackMaxSprintX,homeCenterBackMaxSprintY,homeCenterBackMaxSprintSpeed,homeCenterBackMinSpeedX,homeCenterBackMinSpeedY,homeCenterBackMinSpeed,homeCenterBackMaxSpeedX,homeCenterBackMaxSpeedY,homeCenterBackMaxSpeed,homeRightBackMaxX,homeRightBackMinX,homeRightBackMaxY,homeRightBackMinY,homeRightBackAvgX,homeRightBackAvgY,homeRightBackAvgSpeed,homeRightBackSlowAvgX,homeRightBackSlowAvgY,homeRightBackHirAvgX,homeRightBackHirAvgY,homeRightBackMaxSprintX,homeRightBackMaxSprintY,homeRightBackMaxSprintSpeed,homeRightBackMinSpeedX,homeRightBackMinSpeedY,homeRightBackMinSpeed,homeRightBackMaxSpeedX,homeRightBackMaxSpeedY,homeRightBackMaxSpeed,homeLeftWingerMaxX,homeLeftWingerMinX,homeLeftWingerMaxY,homeLeftWingerMinY,homeLeftWingerAvgX,homeLeftWingerAvgY,homeLeftWingerAvgSpeed,homeLeftWingerSlowAvgX,homeLeftWingerSlowAvgY,homeLeftWingerHirAvgX,homeLeftWingerHirAvgY,homeLeftWingerMaxSprintX,homeLeftWingerMaxSprintY,homeLeftWingerMaxSprintSpeed,homeLeftWingerMinSpeedX,homeLeftWingerMinSpeedY,homeLeftWingerMinSpeed,homeLeftWingerMaxSpeedX,homeLeftWingerMaxSpeedY,homeLeftWingerMaxSpeed,homeCenterMiddleMaxX,homeCenterMiddleMinX,homeCenterMiddleMaxY,homeCenterMiddleMinY,homeCenterMiddleAvgX,homeCenterMiddleAvgY,homeCenterMiddleAvgSpeed,homeCenterMiddleSlowAvgX,homeCenterMiddleSlowAvgY,homeCenterMiddleHirAvgX,homeCenterMiddleHirAvgY,homeCenterMiddleMaxSprintX,homeCenterMiddleMaxSprintY,homeCenterMiddleMaxSprintSpeed,homeCenterMiddleMinSpeedX,homeCenterMiddleMinSpeedY,homeCenterMiddleMinSpeed,homeCenterMiddleMaxSpeedX,homeCenterMiddleMaxSpeedY,homeCenterMiddleMaxSpeed,homeRightWingerMaxX,homeRightWingerMinX,homeRightWingerMaxY,homeRightWingerMinY,homeRightWingerAvgX,homeRightWingerAvgY,homeRightWingerAvgSpeed,homeRightWingerSlowAvgX,homeRightWingerSlowAvgY,homeRightWingerHirAvgX,homeRightWingerHirAvgY,homeRightWingerMaxSprintX,homeRightWingerMaxSprintY,homeRightWingerMaxSprintSpeed,homeRightWingerMinSpeedX,homeRightWingerMinSpeedY,homeRightWingerMinSpeed,homeRightWingerMaxSpeedX,homeRightWingerMaxSpeedY,homeRightWingerMaxSpeed,homeCenterForwardMaxX,homeCenterForwardMinX,homeCenterForwardMaxY,homeCenterForwardMinY,homeCenterForwardAvgX,homeCenterForwardAvgY,homeCenterForwardAvgSpeed,homeCenterForwardSlowAvgX,homeCenterForwardSlowAvgY,homeCenterForwardHirAvgX,homeCenterForwardHirAvgY,homeCenterForwardMaxSprintX,homeCenterForwardMaxSprintY,homeCenterForwardMaxSprintSpeed,homeCenterForwardMinSpeedX,homeCenterForwardMinSpeedY,homeCenterForwardMinSpeed,homeCenterForwardMaxSpeedX,homeCenterForwardMaxSpeedY,homeCenterForwardMaxSpeed,awayDBSCANx,awayDBSCANy,awayMaxX,awayMinX,awayMaxY,awayMinY,awayAvgX,awayAvgY,awayAvgSpeed,awayMinSpeedX,awayMinSpeedY,awayMinSpeed,awayMaxSpeedX,awayMaxSpeedY,awayMaxSpeed,awayMaxSprintX,awayMaxSprintY,awayMaxSprint,awayGoalKeeperX,awayGoalKeeperY,awayGoalKeeperSpeed,awayLeftBackMaxX,awayLeftBackMinX,awayLeftBackMaxY,awayLeftBackMinY,awayLeftBackAvgX,awayLeftBackAvgY,awayLeftBackA

,# match_id,half,min,second,hgkx,hx1,hx2,hx3,hx4,hx5,hx6,hx7,hx8,hx9,hx10,hx11,hx12,hx13,hx14,hx15,hx16,hx17,hx18,hx19,hx20,hgky,hy1,hy2,hy3,hy4,hy5,hy6,hy7,hy8,hy9,hy10,hy11,hy12,hy13,hy14,hy15,hy16,hy17,hy18,hy19,hy20,agkx,ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8,ax9,ax10,ax11,ax12,ax13,ax14,ax15,ax16,ax17,ax18,ax19,ax20,agky,ay1,ay2,ay3,ay4,ay5,ay6,ay7,ay8,ay9,ay10,ay11,ay12,ay13,ay14,ay15,ay16,ay17,ay18,ay19,ay20
5415,60576.00000,2.00000,94.00000,4.00000,3.92188,0.00000,15.32031,15.86719,0.00000,15.65625,49.43750,0.00000,0.00000,24.06250,0.00000,0.00000,0.00000,36.09375,23.26562,0.00000,26.18750,0.00000,19.07812,32.90625,0.00000,34.40625,0.00000,41.25000,34.96875,0.00000,57.43750,44.62500,0.00000,0.00000,49.71875,0.00000,0.00000,0.00000,42.87500,43.25000,0.00000,56.31250,0.00000,27.71875,17.35938,0.00000,82.75000,54.46875,0.00000,39.15625,45.28125,28.25000,13.95312,0.00000,27.07812,0.00000,0.00000,0.00000,14.70312,59.25000,34.68750,0.00000,0.00000,29.75000,0.00000,0.00000,0.00000,34.37500,42.03125,0.00000,52.75000,22.06250,29.96875,34.59375,0.00000,55.93750,0.00000,0.00000,0.00000,57.81250,24.93750,16.76562,0.00000,0.00000,48.09375,0.00000,0.00000,0.00000
5416,60576.00000,2.00000,94.00000,5.00000,4.22656,0.00000,16.75000,17.17188,0.00000,16.45312,48.46875,0.00000,0.00000,23.65625,0.00000,0.00000,0.00000,36.53125,23.81250,0.00000,28.18750,0.00000,18.85938,34.15625,0.00000,34.68750,0.00000,41.50000,34.93750,0.00000,55.06250,45.59375,0.00000,0.00000,49.46875,0.00000,0.00000,0.00000,43.65625,43.18750,0.00000,54.31250,0.00000,27.34375,17.28125,0.00000,82.31250,53.46875,0.00000,37.28125,44.34375,27.84375,14.67188,0.00000,28.95312,0.00000,0.00000,0.00000,16.50000,58.28125,35.84375,0.00000,0.00000,29.65625,0.00000,0.00000,0.00000,34.56250,43.06250,0.00000,51.84375,24.65625,30.82812,34.09375,0.00000,55.37500,0.00000,0.00000,0.00000,52.78125,26.76562,17.26562,0.00000,0.00000,50.03125,0.00000,0.00000,0.00000
5417,60576.00000,2.00000,94.00000,6.00000,4.46875,0.00000,17.90625,18.59375,0.00000,18.42188,48.03125,0.00000,0.00000,23.45312,0.00000,0.00000,0.00000,37.37500,24.51562,0.00000,30.70312,0.00000,19.40625,37.15625,0.00000,34.81250,0.00000,41.56250,34.46875,0.00000,53.90625,47.15625,0.00000,0.00000,48.87500,0.00000,0.00000,0.00000,42.53125,42.78125,0.00000,52.37500,0.00000,26.18750,16.46875,0.00000,82.31250,52.59375,0.00000,36.40625,44.37500,27.56250,15.74219,0.00000,31.67188,0.00000,0.00000,0.00000,21.25000,57.71875,38.59375,0.00000,0.00000,27.12500,0.00000,0.00000,0.00000,34.53125,44.18750,0.00000,48.37500,27.46875,31.76562,33.87500,0.00000,54.03125,0.00000,0.00000,0.00000,49.53125,28.26562,17.23438,0.00000,0.00000,50.28125,0.00000,0.00000,0.00000
5418,60576.00000,2.00000,94.00000,7.00000,4.60547,0.00000,18.50000,19.85938,0.00000,20.35938,47.90625,0.00000,0.00000,23.43750,0.00000,0.00000,0.00000,38.81250,24.75000,0.00000,32.93750,0.00000,20.03125,41.09375,0.00000,34.84375,0.00000,41.78125,33.93750,0.00000,53.59375,48.84375,0.00000,0.00000,48.65625,0.00000,0.00000,0.00000,40.65625,42.46875,0.00000,51.90625,0.00000,24.60938,15.85938,0.00000,82.43750,51.90625,0.00000,35.68750,43.93750,27.35938,17.10938,0.00000,33.75000,0.00000,0.00000,0.00000,24.81250,57.25000,42.34375,0.00000,0.00000,24.75000,0.00000,0.00000,0.00000,34.65625,45.25000,0.00000,44.90625,29.82812,32.46875,33.71875,0.00000,52.09375,0.00000,0.00000,0.00000,45.21875,29.56250,16.82812,0.00000,0.00000,47.87500,0.00000,0.00000,0.00000
5419,60576.00000,2.00000,94.00000,8.00000,4.76953,0.00000,18.57812,20.78125,0.00000,21.67188,47.96875,0.00000,0.00000,24.06250,0.00000,0.00000,0.00000,40.50000,24.54688,0.00000,33.78125,0.00000,20.43750,44.65625,0.00000,35.15625,0.00000,42.06250,33.50000,0.00000,54.71875,51.06250,0.00000,0.00000,48.90625,0.00000,0.00000,0.00000,39.53125,42.15625,0.00000,52.81250,0.00000,23.20312,15.32031,0.00000,82.50000,51.06250,0.00000,35.53125,43.21875,27.01562,18.46875,0.00000,32.81250,0.00000,0.00000,0.00000,27.00000,56.87500,45.96875,0.00000,0.00000,23.54688,0.0000

### Ball Index Data

In [19]:
match_id = 60565
features_df = pd.read_csv('../data/general/feature-set/match{}_features_3.csv'.format(match_id))

ball_idxs = np.sort(features_df['pitch_index'].unique())
print('Ball  index: {}'.format(ball_idxs))

print('Size: {}'.format(len(ball_idxs)))


Ball  index: [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9.]
Size: 10


In [20]:
len(features_df[features_df['pitch_index'] !=0 ])

2141